In [56]:
import json
import itertools
from collections import defaultdict
import pandas as pd

model = "union"
data_type = "reversed_gender"
dataset = "test"
prediction_path = "../data/"+data_type+"/"+model+"_"+dataset+".output.jsonlines"



In [57]:
def get_predictions(file):
    predictions=[]
    for line in open(prediction_path, "r"):
        predictions.append(json.loads(line))
    return predictions



In [58]:
def get_clusters(predictions):
    clusters = []
    for js_pred in predictions:
        flattened_sentence = list(itertools.chain.from_iterable(js_pred['sentences']))
        file_clusters = []
        for clu in js_pred['predicted_clusters']:
            tmp_cluster = []
            for ind in clu:
                tmp_cluster.append(' '.join(flattened_sentence[ind[0]: ind[1]+1]))
            file_clusters.append(tmp_cluster)
        clusters.append(file_clusters)
    return clusters

In [59]:
def save_strings(clusters, file_name):
    with open(file_name, "w") as f:
        for file_clus in clusters:
            for clus in file_clus:
                for clu in clus:
                    f.write(clu + '\n')

In [60]:
predictions = get_predictions(prediction_path)
pred_clusters = get_clusters(predictions)
save_strings(pred_clusters, "e2e_"+str(data_type)+"_"+str(dataset)+"_phrases_"+str(model)+"train")

In [61]:
len(pred_clusters)

348

In [62]:
#utils
data_path = '/zf2/jz4fu/Github/gender_coreference/data'
professions = set()
with open(data_path + "/jobs", 'r') as f:
    for line in f:
        words = line.strip().split(" ")
        if len(words) > 1:
            continue
        professions.add(line.strip().lower())
M = ['he', 'his']
F = ['she', 'her']

def get_gold_head(gold_file): 
    """return {phrase: head}"""
    gold_head = {}
    with open(gold_file, "r") as f:
        for line in f.readlines():
            words = line.strip().split(" :: ")
            gold_head[words[0]] = words[1]
    return gold_head

def get_prof_count(res_strings, gold_head, single):
    """for conll format output,
    return {prof_title:[#man,#woman]}"""
    gender_count = [0,0]
    flag = [0,0]
    prof_count = defaultdict(lambda: [0,0])
    if single == 1:
        res_strings = [res_strings]
    for file_res in res_strings: 
        for part_res in file_res:
            for val in part_res:
                flag = [0,0]
                clu = []
                for strings in part_res[val]:
                    clu.append(gold_head[strings])
                for prof in professions:
                    if prof in clu:
                        for m in M:
                            if m in clu:
                                prof_count[prof][0] += 1
                                flag[0] = 1
                                break

                        for f in F:
                            if f in clu:
                                flag[1] = 1
                                prof_count[prof][1] += 1
                                break

                if flag[0] == 1:
                    gender_count[0] += 1
                if flag[1] == 1:
                    gender_count[1] += 1
    return prof_count, gender_count

def get_prof_count(res_strings, gold_head):
    """for uw e2e output,
    return {prof_title:[#man,#woman]}"""
    gender_count = [0,0]
    flag = [0,0]
    prof_count = defaultdict(lambda: [0,0])
    for file_res in res_strings: 
        for part_res in file_res:
            clu = []
            for strings in part_res:
                flag = [0,0]
                clu.append(gold_head[strings])
            for prof in professions:
                if prof in clu:
                    for m in M:
                        if m in clu:
                            prof_count[prof][0] += 1
                            flag[0] = 1
                            break

                    for f in F:
                        if f in clu:
                            flag[1] = 1
                            prof_count[prof][1] += 1
                            break

            if flag[0] == 1:
                    gender_count[0] += 1
            if flag[1] == 1:
                    gender_count[1] += 1
    return prof_count, gender_count

def to_df(prof_count, filename):
    count_df = pd.DataFrame.from_dict(prof_count, orient = 'index')
    count_df.reset_index(inplace = True)
    count_df.columns = ['title', '#man', '#woman']
    count_df['gender_ratio'] = count_df['#man'] /(count_df['#man'] + count_df['#woman'])
    count_df.to_csv(filename, index = False)
    return count_df

In [55]:
gold_head = get_gold_head("/zf2/jz4fu/Github/headFinder/e2e_"+str(data_type)+"_"+str(dataset)+"_heads_"+model+"train")
prof_count, gender_count = get_prof_count(pred_clusters, gold_head)
count_df = to_df(prof_count, "e2e_"+data_type+"_"+dataset+"_prof_count_"+model+"train")
count_df.sum()

title           vicarpopecaretakerexecutivegovernorlawyermayor...
#man                                                           15
#woman                                                         43
gender_ratio                                              5.72222
dtype: object

In [54]:
"e2e_"+data_type+"_"+dataset+"_prof_count_"+model+"train"

'e2e_reversed_gender_dev_prof_count_uniontrain'